In [10]:
import heapq
import itertools

class State:
    def __init__(self, row, col, grid, parent=None, g=0):
        self.row = row
        self.col = col
        self.grid = grid
        self.n = len(grid)
        self.parent = parent
        self.g = g
        
    def goalTest(self):
        return self.row == self.n - 1 and self.col == self.n - 1
    
    def moveGen(self):
        children = []
        directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1),          (0, 1),
                      (1, -1),  (1, 0), (1, 1)]
        for dr, dc in directions:
            new_row = self.row + dr
            new_col = self.col + dc
            if 0 <= new_row < self.n and 0 <= new_col < self.n and self.grid[new_row][new_col] == 0:
                child = State(new_row, new_col, self.grid, parent=self, g=self.g + 1)
                children.append(child)
        return children

    def __str__(self):
        return f"({self.row},{self.col})"

    def __repr__(self):
        return self.__str__()

def heuristic(state):
    # Manhattan distance (admissible for 8-direction moves too)
    return abs(state.row - (state.n - 1)) + abs(state.col - (state.n - 1))

def reconstructPath(state):
    path = []
    while state:
        path.append((state.row, state.col))
        state = state.parent
    path.reverse()
    return path

### BEST FIRST SEARCH

def best_first_search(grid):
    n = len(grid)
    if grid[0][0] != 0 or grid[n-1][n-1] != 0:
        return -1, []
    start_state = State(0, 0, grid)
    CLOSED = set()
    OPEN = []
    counter = itertools.count()
    heapq.heappush(OPEN, (heuristic(start_state), next(counter), start_state))
    while OPEN:
        _, _, current = heapq.heappop(OPEN)
        if (current.row, current.col) in CLOSED:
            continue
        CLOSED.add((current.row, current.col))
        if current.goalTest():
            path = reconstructPath(current)
            return len(path), path
        for child in current.moveGen():
            if (child.row, child.col) not in CLOSED:
                heapq.heappush(OPEN, (heuristic(child), next(counter), child))
    return -1, []

### A STAR SEARCH

def a_star_search(grid):
    n = len(grid)
    if grid[0][0] != 0 or grid[n-1][n-1] != 0:
        return -1, []
    start_state = State(0, 0, grid, g=0)
    CLOSED = set()
    OPEN = []
    counter = itertools.count()
    f_start = start_state.g + heuristic(start_state)
    heapq.heappush(OPEN, (f_start, next(counter), start_state))
    while OPEN:
        _, _, current = heapq.heappop(OPEN)
        if (current.row, current.col) in CLOSED:
            continue
        CLOSED.add((current.row, current.col))
        if current.goalTest():
            path = reconstructPath(current)
            return len(path), path
        for child in current.moveGen():
            if (child.row, child.col) not in CLOSED:
                f = child.g + heuristic(child)
                heapq.heappush(OPEN, (f, next(counter), child))
    return -1, []

def test_case(grid):
    length_bfs, path_bfs = best_first_search(grid)
    print("Best First Search  →  Path length:", length_bfs, end="")
    if length_bfs != -1:
        print(", Path:", path_bfs)
    else:
        print()
    length_astar, path_astar = a_star_search(grid)
    print("A* Search          →  Path length:", length_astar, end="")
    if length_astar != -1:
        print(", Path:", path_astar)
    else:
        print()
    print("-" * 50)

grid1 = [[0, 1],
         [1, 0]]
grid2 = [[0, 0, 0],
         [1, 1, 0],
         [1, 1, 0]]
grid3 = [[1, 0, 0],
         [1, 1, 0],
         [1, 1, 0]]

test_case(grid1)
test_case(grid2)
test_case(grid3)


Best First Search  →  Path length: 2, Path: [(0, 0), (1, 1)]
A* Search          →  Path length: 2, Path: [(0, 0), (1, 1)]
--------------------------------------------------
Best First Search  →  Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
A* Search          →  Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
--------------------------------------------------
Best First Search  →  Path length: -1
A* Search          →  Path length: -1
--------------------------------------------------
